# Heat equation

## imports

In [1]:
import numpy as np
import pandas as pd
import scipy

In [2]:
import matplotlib.pyplot as plt

## initialization

In [3]:
X_max = 120
Y_max = 70
X_min = Y_min = 0

NUMBER_X = 120
NUMBER_Y = 70

In [7]:
X_RANGE = np.linspace(X_min, X_max, NUMBER_X)
Y_RANGE = np.linspace(Y_min, Y_max, NUMBER_Y)

X_STEP = X_RANGE[1] - X_RANGE[0]
Y_STEP = Y_RANGE[1] - Y_RANGE[0]

In [47]:
X, Y = np.meshgrid(X_RANGE, Y_RANGE, indexing='ij')

T = np.zeros_like(X)
source = np.zeros_like(X) 

## boundary conditions

In [50]:
T_LOWER = 0         # lower bound
T_UPPER = 0         # upper bound
Q_LEFT = 0          # left bound
T_RIGHT = 0         # right bound

## main procedure

## vizualization